In [1]:
import os
import re
import random
import numpy as np
import pandas as pd
from datasets import load_dataset, load_from_disk, concatenate_datasets, DatasetDict

## Load datasets from multiple source

In [5]:
# load base dataset
dataset = load_dataset('../data/alpaca-cleaned') # remember to switch to your own path

HF google storage unreachable. Downloading and preparing it from source


Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
# other data source
dataset_chat = load_dataset('../data/ChatAlpaca-20k')
dataset_hhrlhf= load_dataset('../data/hh-rlhf', split='train')

## Format other data source to Alpaca

In [14]:
def format_chat_to_alpaca(example):
    chat_text = example['messages']
    text = ''
    for e in chat_text:
        if e['role'] == 'user':
            text += '\n###Question: ' + e['content']
        else:
            text += '\n###Answer: ' + e['content']
    text = text[1:]
    instruction = text.rsplit('\n###Answer:',1)[0]
    instruction = instruction.split('###Question: ',1)[1]
    output = text.rsplit('\n###Answer: ',1)[1]

    example['input'] = ''
    example['output'] = output
    example['instruction'] = instruction

    return example

In [15]:
dataset_chat = dataset_chat['train'].map(format_chat_to_alpaca, num_proc=8, remove_columns = ["id", "messages"])

In [16]:
def format_hhrlhf_to_alpaca(example):
    '''
    alpaca format : {instruction}, {input}, {output}
    hh rlhf format : {chosen}, {rejected}
        chosen: \n\nHuman: 11\n\nAssistant: 22 \n\nHuman: 33\n\nAssistant: 44
    target format is
        chosen: \n###Question: 11\n###Answer: 22 \n### Question 33\n###Answer: 44
        -> instruction: 【11\n###Answer: 22 \n### Question 33】
        -> output: 【 44】
    '''
    text = example["chosen"]
    text = re.sub(r'\n\nHuman:', '\n###Question:', text)
    text = re.sub(r'\n\nAssistant:', '\n###Answer:', text)
    text = text[1:]

    instruction = text.rsplit('\n###Answer:',1)[0]
    instruction = instruction.split('###Question: ',1)[1]
    output = text.rsplit('\n###Answer: ',1)[1]

    example['input'] = ''
    example['output'] = output
    example['instruction'] = instruction

    return example

In [17]:
dataset_hhrlhf = dataset_hhrlhf.map(format_hhrlhf_to_alpaca, num_proc=8, remove_columns = ["chosen", "rejected"])

Map (num_proc=8):   0%|          | 0/160800 [00:00<?, ? examples/s]

### Data Concatenation

In [18]:
dataset = concatenate_datasets([dataset['train'], dataset_hhrlhf, dataset_chat])
dataset = DatasetDict({'train': dataset})
dataset = dataset.shuffle(seed=42)

In [19]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 232160
    })
})
